# Julia Package Development Tutorial
After this presentation, you will know how to:
1. initiate a package and connect it to GitHub
2. define, test and document functions
3. add dependencies and increase complexity
4. deploy documentation online

---

## Initiate a package

It might be time-consuming to build your own architecture from scratch, but thanks to PkgTemplates.jl this procedure is automatic. Also, it is good practice to generate the draft package in an empty directory and then you can import the content that you've already written.

There is a bunch of additional plugins and features that you can insert in your package draft, such as CI and documentation, but I suggest that you don't overdo with them, because they make it more difficult to find your way among the many generated files. More often than not I just create a basic template and then add more features manually.

In [ ]:
# This is the Julia REPL

using PkgTemplates

t = Template(;
        dir = @__DIR__,
        plugins = [
                License(; name = "MIT"),
                Codecov(),
                GitHubActions(),
                Documenter{GitHubActions}(),
                ],
        )

t("MyPkg")

Create a new repository on GitHub and call it with the same name as your local project. The remote should already be set up, so you can make the connection by pushing the local to the master branch.

In [ ]:
# This is the command line

# push local to master
git push origin master

## Define, test and document functions
Navigate to `src/MyPkg.jl` and look into it. This file is the module, where you can define the functions that will be exported with your package. The triple quotes upstream each function are called docstrings and their content will appear in the documentation.

In [ ]:
# This is src/MyPkg.jl

module MyPkg

"""
    my_sum(a::Int64, b::Int64)

Adds two integers together.
"""
my_sum(a::Int64, b::Int64) = a + b

"""
    my_product(a::Float64, b::Float64)

Multiply two floats together.
"""
my_product(a::Float64, b::Float64) = a * b

end

Tests can be found in the test directory. The package template has already prepared the draft in runtests.jl that you can fill with @test macros, which check if the passed conditionals are true or false.

In [6]:
# This is test/runtests.jl

using Test
using MyPkg

@testset "MyPkg.jl" begin

    a = my_sum(2, 2)
    b = my_product(2.0, 2.0)

    @test a == b

end

Test Summary: | Pass  Total
MyPkg.jl      |    1      1


Test.DefaultTestSet("MyPkg.jl", Any[], 1, false, false)

### How to write meaningful tests?
Different levels of complexity for your tests:
* *definition level*: I'm happy as long as this variable is defined
* *equality level*: I can expect A to be the exact copy of B
* *partial-equality level*: I can expect A to be more or less equal to B
* *absolute-difference level*: results are highly variable (+/-, order of magnitude), but their absolute difference is always less than some value

In [ ]:
# This is MicrobiomeAnalysis/test/test_transform.jl

# is some variable defined?
@test @isdefined Xapp

# is each element in A equal to each element in B?
@test z_assay1 == z_assay2

# are the rounded means equal to each other up to a tolerance of 15 digits?
@test round(mean(relabund_assay), digits = 15) == round(mean(assay(se, "relabund_assay")), digits = 15)

# is the absolute value of the difference between the means less than the tolerance?
@test abs(mean(clr_assay1) - mean(clr_assay2)) < 10e-16

## Add dependencies and increase complexity
Imagine that one of your functions depends on some other package. You'll have to import it.

In [ ]:
# This is added to src/MyPkg.jl

using Statistics: mean
# alternatively (with some differences)
# import Statistics: mean

"""
    my_mean(a::Real, b::Real)

Takes a strange mean.
"""
my_mean(a::Real, b::Real) = mean(a, b) + 1

All of the above works as a good temporary solution, as long as you don't need many external utilites or multiple source scripts. In such case, you might want to move your functions to separate scripts and `include` them in the module, which will trigger their execution.

In [ ]:
# This is src/MyPkg.jl

module MyPkg

using Statistics: mean
export my_sum, my_product, my_mean

include("funcs.jl")

end

In [ ]:
# This is src/funcs.jl

"""
    my_sum(a::Int64, b::Int64)

Adds two integers together.
"""
my_sum(a::Int64, b::Int64) = a + b

"""
    my_product(a::Float64, b::Float64)

Multiply two floats together.
"""
my_product(a::Float64, b::Float64) = a * b

"""
    my_mean(a::Real, b::Real)

Takes a strange mean.
"""
my_mean(a::Real, b::Real) = mean(a, b) + 1

## Deploy documentation online
Commit the changes to master. After `.github/workflows/CI.yml` has run, a new branch "gh-pages" should be automatically created. You might have to go the repo settings and manually set gh-pages as the default branch for deployment. From now on, every time you push a new tag, the current version of the docs will be rendered at "https://MyOrg.github.io/MyPkg.jl/stable/", whereas "https://MyOrg.github.io/MyPkg.jl/dev/" will always show the development version (latest) no matter what.

In [ ]:
# This is the command line

# commit changes locally
git commit -m "Add new features"
# create a new tag locally
git tag -a v0.1.0 -m "Release version 0.1.0"
# push changes to origin/master
git push origin master
# push tags to origin
git push --tag